In [1]:
import os
import sys
import numpy as np
sys.path.insert(0, "/home/bnapp/arivkindNet/wisdrc/")
from parse_commandline import parse_commandline
from utils.imagenet_dataset import get_dataset
from utils.unprocess import *
from utils.image_utils import *
from utils.image_processing import rescale
from utils.split_keras_model import split_model
from models.student_models import full_size_rggb_fe
import tensorflow as tf
import matplotlib.pyplot as plt

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
def enforce_list(x):
    if type(x) is list:
        return x
    else:
        return [x]


def squeeze_list(x):
    if len(x) == 1:
        return x[0]
    else:
        return x

In [3]:
teacher = keras.models.load_model('../../pretrained_models/Alexnetmodel')
split_after_layer =  'pool1'


# teacher = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3),
#                                                       include_top=True,
#                                                       weights='imagenet')
# split_after_layer = 'block_1_depthwise_BN'

2023-05-09 11:29:59.076330: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1917] Ignoring visible gpu device (device: 1, name: NVIDIA GeForce GT 730, pci bus id: 0000:65:00.0, compute capability: 3.5) with core count: 2. The minimum required count is 8. You can adjust this requirement with the env var TF_MIN_GPU_MULTIPROCESSOR_COUNT.
2023-05-09 11:29:59.483373: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10247 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:17:00.0, compute capability: 6.1


In [4]:
teacher.summary()

Model: "Alex"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 57, 57, 96)        34944     
_________________________________________________________________
norm1 (Lambda)               (None, 57, 57, 96)        0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 28, 28, 96)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 28, 28, 256)       307456    
_________________________________________________________________
norm2 (Lambda)               (None, 28, 28, 256)       0         
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 13, 13, 256)       0         
_________________________________________________________________
conv3 (Conv2D)               (None, 13, 13, 384)       885120 

In [5]:
m1,m2 = split_model(teacher, split_after_layer)

In [9]:
m1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 57, 57, 96)        34944     
_________________________________________________________________
norm1 (Lambda)               (None, 57, 57, 96)        0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 28, 28, 96)        0         
Total params: 34,944
Trainable params: 34,944
Non-trainable params: 0
_________________________________________________________________


In [10]:
m2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_top (InputLayer)       [(None, 28, 28, 96)]      0         
_________________________________________________________________
conv2 (Conv2D)               (None, 28, 28, 256)       307456    
_________________________________________________________________
norm2 (Lambda)               (None, 28, 28, 256)       0         
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 13, 13, 256)       0         
_________________________________________________________________
conv3 (Conv2D)               (None, 13, 13, 384)       885120    
_________________________________________________________________
conv4 (Conv2D)               (None, 13, 13, 384)       663936    
_________________________________________________________________
conv5 (Conv2D)               (None, 13, 13, 256)       442624

In [11]:
uu1 = m1.predict(np.random.normal(size=(16,227,227,3)))
uu2 = m2.predict(np.random.normal(size=(16,28,28,96)))


2023-05-09 12:44:22.376715: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2023-05-09 12:44:23.067275: I tensorflow/stream_executor/cuda/cuda_dnn.cc:381] Loaded cuDNN version 8300


In [12]:
uu1

array([[[[3.30637187e-01, 1.43391776e+00, 8.86386514e-01, ...,
          2.46378139e-01, 1.58128142e-03, 0.00000000e+00],
         [7.02897608e-01, 1.50635672e+00, 1.03973210e+00, ...,
          6.18838012e-01, 0.00000000e+00, 0.00000000e+00],
         [7.02897608e-01, 2.88170624e+00, 1.03973210e+00, ...,
          0.00000000e+00, 0.00000000e+00, 5.69788739e-02],
         ...,
         [1.00741708e+00, 3.40101552e+00, 1.18493629e+00, ...,
          1.53925598e-01, 2.10335851e-02, 0.00000000e+00],
         [4.02279377e-01, 2.25948429e+00, 1.05860937e+00, ...,
          3.50753993e-01, 2.10335851e-02, 0.00000000e+00],
         [5.58739662e-01, 2.96821642e+00, 1.05860937e+00, ...,
          1.63856834e-01, 1.01269782e-02, 0.00000000e+00]],

        [[5.40990353e-01, 2.82052541e+00, 1.32175016e+00, ...,
          2.46378139e-01, 2.09098458e-02, 0.00000000e+00],
         [1.92646539e+00, 3.86539578e+00, 6.54688418e-01, ...,
          6.18838012e-01, 0.00000000e+00, 0.00000000e+00],
        

In [13]:
uu2

array([[0.00073419, 0.00112603, 0.00109277, ..., 0.00043909, 0.00088262,
        0.00291523],
       [0.00075908, 0.0010248 , 0.00118749, ..., 0.00041054, 0.000875  ,
        0.00288224],
       [0.00080428, 0.00112449, 0.00113596, ..., 0.00041055, 0.0009835 ,
        0.00314751],
       ...,
       [0.00083853, 0.0010727 , 0.0011367 , ..., 0.00046142, 0.00099822,
        0.00321871],
       [0.00081749, 0.00113056, 0.00123249, ..., 0.00041388, 0.0009293 ,
        0.00266921],
       [0.0007968 , 0.00109019, 0.00116515, ..., 0.00038682, 0.00088414,
        0.00297334]], dtype=float32)

# PLAYGROUND

In [6]:
source_model, last_bottom_layer_name = teacher, split_after_layer
silent=False
custom_objects=None
layers_by_output_tensor_name = {}

DUMMY_INSTANCE = 999
# #taking care of a case that input layer was not defined as a keras layer explicitly
# if type(source_model.layers[0]) != keras.layers.InputLayer:
#     skip_input_flag = True
#     input_bottom = keras.layers.Input(shape=source_model.layers[0].input.shape.as_list()[1:],
#                                name='input_top')

for layer in source_model.layers:
    if not silent:
        print(layer.name)
    config = layer.get_config()
    layers_by_output_tensor_name[layer.output.name] = {'name': layer.name,
                                                       'layer_instance': keras.layers.deserialize(
                                                           {'class_name': layer.__class__.__name__,
                                                            'config': config}, custom_objects=custom_objects),
                                                       'input_tensor_names': [uu.name for uu in
                                                                              enforce_list(layer.input)],
                                                       'output_tensor_instance': None}

# out_tensor = layer_instance(*input_tensors_names)
last_bottom_layer = source_model.get_layer(last_bottom_layer_name)
input_top = keras.layers.Input(shape=last_bottom_layer.output.shape.as_list()[1:],
                               name='input_top')

#taking care of a case that input layer was not defined as a keras layer explicitly
layers_by_output_tensor_name[source_model.layers[0].output.name]['output_tensor_instance'] = DUMMY_INSTANCE
#the connection between layers in now broken at the input layer of the new model
layers_by_output_tensor_name[last_bottom_layer.output.name]['output_tensor_instance'] = input_top



conv1
norm1
pool1
conv2
norm2
pool2
conv3
conv4
conv5
pool5
flatten
fc6
fc7
fc8
prob


In [7]:
    for ii in range(len(layers_by_output_tensor_name)):
        if not silent:
            print('iteration', ii)
        for tensor_name in layers_by_output_tensor_name:
            if layers_by_output_tensor_name[tensor_name]['output_tensor_instance'] is None:
                not_ready_yet = False
                for input_tensor in layers_by_output_tensor_name[tensor_name]['input_tensor_names']:
                    if layers_by_output_tensor_name[input_tensor]['output_tensor_instance'] is None:
                        not_ready_yet = True
                if not not_ready_yet:
                    inputs = [layers_by_output_tensor_name[input_tensor]['output_tensor_instance'] for input_tensor in
                              layers_by_output_tensor_name[tensor_name]['input_tensor_names']]
                    if squeeze_list(inputs) is not DUMMY_INSTANCE:
                        layers_by_output_tensor_name[tensor_name]['output_tensor_instance'] = \
                            layers_by_output_tensor_name[tensor_name]['layer_instance'](squeeze_list(inputs))
                    if not silent:
                        print('connected layer:', layers_by_output_tensor_name[tensor_name]['name'])

iteration 0
connected layer: norm1
connected layer: conv2
connected layer: norm2
connected layer: pool2
connected layer: conv3
connected layer: conv4
connected layer: conv5
connected layer: pool5
connected layer: flatten
connected layer: fc6
connected layer: fc7
connected layer: fc8
connected layer: prob
iteration 1
connected layer: norm1
iteration 2
connected layer: norm1
iteration 3
connected layer: norm1
iteration 4
connected layer: norm1
iteration 5
connected layer: norm1
iteration 6
connected layer: norm1
iteration 7
connected layer: norm1
iteration 8
connected layer: norm1
iteration 9
connected layer: norm1
iteration 10
connected layer: norm1
iteration 11
connected layer: norm1
iteration 12
connected layer: norm1
iteration 13
connected layer: norm1
iteration 14
connected layer: norm1


In [8]:
layers_by_output_tensor_name[tensor_name]['layer_instance'](squeeze_list(inputs))

InvalidArgumentError: Value for attr 'T' of int32 is not in the list of allowed values: half, bfloat16, float, double
	; NodeDef: {{node Softmax}}; Op<name=Softmax; signature=logits:T -> softmax:T; attr=T:type,allowed=[DT_HALF, DT_BFLOAT16, DT_FLOAT, DT_DOUBLE]> [Op:Softmax]

In [ ]:
squeeze_list(inputs) is DUMMY_INSTANCE

In [ ]:
inputs

In [ ]:
layers_by_output_tensor_name[tensor_name]['layer_instance']

In [ ]:
squeeze_list(inputs)

In [ ]:
inputs

In [ ]:
layers_by_output_tensor_name.keys()

In [ ]:
input_tensor

In [ ]:
tensor_name

In [ ]:
layers_by_output_tensor_name

In [ ]:
source_model.inputs[0]

In [ ]:
source_model.layers[0].input

In [ ]:
source_model.inputs[0] is source_model.layers[0].input

In [ ]:
source_model.layers[0]

In [ ]:
type(source_model.layers[0])

In [ ]:
type(source_model.layers[0]) != keras.layers.InputLayer

In [ ]:
source_model.layers[0].get_config()

In [ ]:
source_model.layers[0].input.shape.as_list()[1:]

In [ ]:
layer = source_model.layers[0].input

In [ ]:
    config = layer.get_config()
    layers_by_output_tensor_name[layer.output.name] = {'name': layer.name,
                                                       'layer_instance': keras.layers.deserialize(
                                                           {'class_name': layer.__class__.__name__,
                                                            'config': config}, custom_objects=custom_objects),
                                                       'input_tensor_names': [uu.name for uu in
                                                                              enforce_list(layer.input)],
                                                       'output_tensor_instance': None}